In [1]:
#!pip install --upgrade metagpt

In [1]:
#!metagpt --init-config 

In [3]:
with open('config2.yaml', 'w') as file:
    file.write("""llm:
  api_type: 'zhipuai'
  api_key: 
  model: 'glm-4-plus'
""")

In [ ]:
!cp config2.yaml ~/.metagpt/config2.yaml
!cat ~/.metagpt/config2.yaml

# 3 agents

In [26]:
import asyncio

from metagpt.actions import Action, UserRequirement
from metagpt.logs import logger
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.environment import Environment

from metagpt.const import MESSAGE_ROUTE_TO_ALL
# 加载环境变量
from dotenv import load_dotenv 
load_dotenv()

classroom = Environment()

class WriteStructure(Action):
    """
    学生Agent的撰写作文大纲Action。
    """
    name: str = "WriteOutline"

    PROMPT_TEMPLATE: str = """
您现在是短篇小说领域经验丰富的小说作家。我们需要您创作出游戏主题小说的目录和章节内容概况。
请按照以下要求提供该小说的具体目录和目录中的故事概况：
1. 输出必须严格符合指定语言。
2. 回答必须严格按照字典格式，如: {"题目": "xxx", "目录": {"第一章：章节标题": "故事概况", "第二章：章节标题": "故事概况"}} 。
3. 目录应尽可能引人注目和充分，包括一级目录和本章故事概况。
4. 不要有额外的空格或换行符。
"""
 

    async def run(self, msg: str):
        """
        根据用户提供的主题撰写一篇小说大纲。
        """
        prompt = self.PROMPT_TEMPLATE.format()#msg=msg

        rsp = await self._aask(prompt)

        return rsp

class WriteAction(Action):
    """
    学生Agent的撰写作文Action。
    """
    name: str = "WriteEssay"

    PROMPT_TEMPLATE: str = """
    这里是历史对话记录：{msg}。
    请你根据用户提供的主题撰写一篇小说，只返回生成的作文内容，不包含其他文本。
    如果老师提供了关于小说的建议，请根据建议修改你的历史小说并返回。
    小说字数不少于10000字。
    你的小说如下：
    """

    async def run(self, msg: str):
        """
        根据用户提供的主题撰写一篇小说，并在收到老师的修改建议后进行修改。
        """
        prompt = self.PROMPT_TEMPLATE.format(msg=msg)

        rsp = await self._aask(prompt)

        return rsp

class ReviewAction(Action):
    """
    老师Agent的审阅作文Action。
    """
    name: str = "ReviewEssay"

    PROMPT_TEMPLATE: str = """
    这里是历史对话记录：{msg}。
    你是一名老师，现在请检查学生创作的关于用户提供的主题的小说，并给出你的修改建议。你更喜欢逻辑清晰的结构和有趣的口吻。
    只返回你的修改建议，不要包含其他文本。
    你的修改建议如下：
    """

    async def run(self, msg: str):
        """
        审阅学生的小说，并给出修改建议。
        """
        prompt = self.PROMPT_TEMPLATE.format(msg=msg)

        rsp = await self._aask(prompt)

        return rsp
class WriteStructure(Action):
    """
    学生Agent的撰写作文大纲Action。
    """
    name: str = "WriteOutline"

    PROMPT_TEMPLATE: str = """
您现在是短篇小说领域经验丰富的小说作家。我们需要您创作出{msg}主题小说的标题、目录和章节内容概况。
请按照以下要求提供该小说的具体目录和目录中的故事概况：
1. 输出必须严格符合指定语言。
2. 回答必须严格按照字典格式罗列题目和各章节内容。
3. 目录应尽可能引人注目和充分，包括一级目录和本章故事概况。
4. 不要有额外的空格或换行符。
"""
#，如: {"题目": "xxx", "目录": {"第一章：章节标题": "故事概况", "第二章：章节标题": "故事概况"}} 。

    async def run(self, msg: str):
        """
        根据用户提供的主题撰写一篇小说大纲。
        """
        prompt = self.PROMPT_TEMPLATE.format(msg=msg)#msg=msg

        rsp = await self._aask(prompt)

        return rsp


class Oulinewriter(Role):
    """
    大纲编写者角色。
    """
    name: str = "Apple"
    profile: str = "Oulinewriter"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([WriteStructure])  # 设置学生的动作为撰写作文
        #self._watch([UserRequirement])  # 监听用户要求和老师的审阅动作

    async def _act(self) -> Message:
        """
        大纲编写者动作：根据用户要求撰写小说大纲。
        """
        logger.info(f"{self._setting}: ready to {self.rc.todo}")
        todo = self.rc.todo

        msg = self.get_memories()
        #print(msg)# 获取所有对话记忆
        logger.info(msg)
        essay_text = await WriteStructure().run(msg)
        logger.info(f'outlinewriter : {essay_text}')
        msg = Message(content=essay_text, role=self.profile, cause_by=type(todo))

        return msg
async def main():
    msg = "游戏"
    role = Oulinewriter()
    logger.info(msg)
    result = await role.run(msg)
    logger.info(result)
#asyncio.run(main())
g = main() # 此时的函数是异步协程函数，此时函数执行得到一个协程对象
#asyncio.run(g)
await g
class Student(Role):
    """
    学生角色。
    """
    name: str = "cheems"
    profile: str = "Student"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([WriteAction])  # 设置学生的动作为撰写作文
        self._watch([UserRequirement,WriteStructure, ReviewAction])  # 监听用户要求和老师的审阅动作

    async def _act(self) -> Message:
        """
        学生动作：根据小说大纲撰写完整小说或根据老师的修改建议修改小说。
        """
        logger.info(f"{self._setting}: ready to {self.rc.todo}")
        todo = self.rc.todo

        msg = self.get_memories()  # 获取所有对话记忆
        # logger.info(msg)
        essay_text = await WriteAction().run(msg)
        logger.info(f'student : {essay_text}')
        msg = Message(content=essay_text, role=self.profile, cause_by=type(todo))

        return msg

class Teacher(Role):
    """
    老师角色。
    """
    name: str = "laobai"
    profile: str = "Teacher"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([ReviewAction])  # 设置老师的动作为审阅作文
        self._watch([WriteStructure, ReviewAction])  # 监听学生的撰写作文动作

    async def _act(self) -> Message:
        """
        老师动作：审阅学生的作文并给出修改建议。
        """
        logger.info(f"{self._setting}: ready to {self.rc.todo}")
        todo = self.rc.todo

        msg = self.get_memories()  # 获取所有对话记忆
        review_text = await ReviewAction().run(msg)
        logger.info(f'teacher : {review_text}')
        msg = Message(content=review_text, role=self.profile, cause_by=type(todo))

        return msg



async def main(topic: str, n_round=5):
    """
    运行函数，用户输入一个主题，并将主题发布在环境中，然后运行环境。
    """
    classroom.add_roles([Oulinewriter(),Student(), Teacher()])  # 向环境中添加学生和老师角色

    classroom.publish_message(
        Message(role="Human", content=topic, cause_by=UserRequirement,
                send_to='' or MESSAGE_ROUTE_TO_ALL),
        peekable=False,
    )
    # 发布一条消息，包含用户输入的主题，并将其发送给所有角色

    while n_round > 0:
        # self._save()
        n_round -= 1
        logger.debug(f"max {n_round=} left.")  # 输出剩余对话轮数

        await classroom.run()  # 运行环境
    return classroom.history  # 返回对话历史记录

g=main(topic='游戏')  # 运行主函数，输入主题为 "道德和法律的限制范围"
await g


2024-11-28 13:58:17.079 | INFO     | __main__:main:145 - 游戏
2024-11-28 13:58:17.082 | INFO     | __main__:_act:131 - Apple(Oulinewriter): ready to WriteStructure
2024-11-28 13:58:17.083 | INFO     | __main__:_act:136 - [user: 游戏]


```json
{
  "标题": "虚拟幻境：游戏之巅",
  "目录": [
    {
      "章节": "第一章：觉醒的代码",
      "故事概况": "主角李明在现实生活中遭遇挫折，偶然间获得一款神秘游戏的邀请码，进入了一个名为“幻境”的虚拟世界，开始了他的冒险之旅。"
    },
    {
      "章节": "第二章：初探幻境",
      "故事概况": "李明在游戏中创建角色，结识了新手村的小伙伴们，并完成了第一个任务，逐渐适应了幻境的规则和玩法。"
    },
    {
      "章节": "第三章：隐藏的真相",
      "故事概况": "在一次偶然的任务中，李明发现游戏背后隐藏着不为人知的秘密，似乎与现实世界有着千丝万缕的联系。"
    },
    {
      "章节": "第四章：联盟与背叛",
      "故事概况": "为了揭开真相，李明与几位志同道合的玩家组建了联盟，但在关键时刻，联盟内部出现了背叛者，局势变得复杂。"
    },
    {
      "章节": "第五章：逆风翻盘",
      "故事概况": "面对重重困难，李明凭借智慧和勇气，带领团队逆转局势，成功击败了背叛者，并揭露了游戏公司的阴谋。"
    },
    {
      "章节": "第六章：现实与虚拟的交汇",
      "故事概况": "李明发现游戏中的技能和经验可以带回现实世界，开始利用这些能力解决现实中的问题，生活发生了翻天覆地的变化。"
    },
    {
      "章节": "第七章：终极对决",
      "故事概况": "游戏公司发现了李明的异常，派出最强NPC追杀他。李明在生死关头，激发了潜能，与NPC展开了一场惊心动魄的终极对决。"
    },
    {
      "章节": "第八章：新的起点",
      "故事概况": "经过一系列的冒险和战斗，李明终于揭开了幻境的全部秘密，并找到了平衡现实与虚拟的方法，迎来了新的生活篇章。"
    }
 

2024-11-28 13:58:30.162 | WARNING  | metagpt.utils.cost_manager:update_cost:49 - Model glm-4-plus not found in TOKEN_COSTS.
2024-11-28 13:58:30.163 | INFO     | __main__:_act:138 - outlinewriter : ```json
{
  "标题": "虚拟幻境：游戏之巅",
  "目录": [
    {
      "章节": "第一章：觉醒的代码",
      "故事概况": "主角李明在现实生活中遭遇挫折，偶然间获得一款神秘游戏的邀请码，进入了一个名为“幻境”的虚拟世界，开始了他的冒险之旅。"
    },
    {
      "章节": "第二章：初探幻境",
      "故事概况": "李明在游戏中创建角色，结识了新手村的小伙伴们，并完成了第一个任务，逐渐适应了幻境的规则和玩法。"
    },
    {
      "章节": "第三章：隐藏的真相",
      "故事概况": "在一次偶然的任务中，李明发现游戏背后隐藏着不为人知的秘密，似乎与现实世界有着千丝万缕的联系。"
    },
    {
      "章节": "第四章：联盟与背叛",
      "故事概况": "为了揭开真相，李明与几位志同道合的玩家组建了联盟，但在关键时刻，联盟内部出现了背叛者，局势变得复杂。"
    },
    {
      "章节": "第五章：逆风翻盘",
      "故事概况": "面对重重困难，李明凭借智慧和勇气，带领团队逆转局势，成功击败了背叛者，并揭露了游戏公司的阴谋。"
    },
    {
      "章节": "第六章：现实与虚拟的交汇",
      "故事概况": "李明发现游戏中的技能和经验可以带回现实世界，开始利用这些能力解决现实中的问题，生活发生了翻天覆地的变化。"
    },
    {
      "章节": "第七章：终极对决",
      "故事概况": "游戏公司发现了李明的异常，派出最强NPC追杀他。李明在生死关头，激发了潜能，与NPC展开了一场惊心动魄的终极对决。"
    },
    {
      "章节": "第

 ]
}
```


2024-11-28 13:58:30.264 | INFO     | __main__:_act:168 - cheems(Student): ready to WriteAction


...

世界的曙光。


2024-11-28 13:59:52.797 | INFO     | __main__:_act:168 - cheems(Student): ready to WriteAction
2024-11-28 13:59:52.813 | INFO     | __main__:_act:195 - laobai(Teacher): ready to ReviewAction
2024-11-28 13:59:55.604 | WARNING  | metagpt.utils.common:wrapper:649 - There is a exception in role's execution, in order to resume, we delete the newest role communication message in the role's memory.


Exception: Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/metagpt/utils/common.py", line 640, in wrapper
    return await func(self, *args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/metagpt/roles/role.py", line 550, in run
    rsp = await self.react()
  File "/opt/conda/lib/python3.10/site-packages/metagpt/roles/role.py", line 517, in react
    rsp = await self._react()
  File "/opt/conda/lib/python3.10/site-packages/metagpt/roles/role.py", line 463, in _react
    rsp = await self._act()
  File "/tmp/ipykernel_30/1928168235.py", line 199, in _act
    review_text = await ReviewAction().run(msg)
  File "/tmp/ipykernel_30/1928168235.py", line 85, in run
    rsp = await self._aask(prompt)
  File "/opt/conda/lib/python3.10/site-packages/metagpt/actions/action.py", line 93, in _aask
    return await self.llm.aask(prompt, system_msgs)
  File "/opt/conda/lib/python3.10/site-packages/metagpt/provider/base_llm.py", line 150, in aask
    rsp = a

**第一章：幻界迷踪**

公元2045年，科技的发展已经超越了人类的想象。虚拟现实技术不再是遥不可及的梦想，而是成为了人们日常生活的一部分。在这个时代，最受欢迎的虚拟现实游戏名为“幻界”。

“幻界”是一款全球联网的大型多人在线角色扮演游戏（MMORPG），它的独特之处在于其高度逼真的虚拟世界和几乎无限制的自由度。玩家可以通过神经接口设备进入游戏，体验一个完全不同的平行世界。

李明，一个普通的大学生，也是“幻界”的忠实玩家。他从小就对游戏充满了热情，尤其是对那些能够让他逃离现实、沉浸其中的虚拟世界。今天，他像往常一样，戴上了神经接口头盔，准备进入“幻界”。

“欢迎来到幻界，勇士！”系统的提示音响起，李明的眼前一黑，随后便置身于一个充满奇幻色彩的世界。他站在一片广袤的草原上，远处是巍峨的山脉和神秘的森林，天空中飘浮着几朵洁白的云。

李明操控着自己的角色，一个身穿银色铠甲的骑士，开始了今天的冒险。他的目标是完成一个名为“龙之宝藏”的任务，这个任务据说奖励丰厚，但也异常危险。

在前往任务地点的路上，李明遇到了各种各样的玩家和NPC（非玩家角色）。有的玩家热情地与他打招呼，有的则冷眼相对，甚至还有人试图抢夺他的装备。李明小心翼翼地应对着，他知道在这个世界里，信任和警惕同样重要。

终于，李明来到了任务地点——一座被浓雾笼罩的古老城堡。城堡的大门紧闭，上面刻满了神秘的符文。李明深吸一口气，推开了大门，一股阴冷的气息扑面而来。

城堡内部昏暗而寂静，只有墙壁上摇曳的火把发出微弱的光芒。李明小心翼翼地走在长长的走廊上，耳边传来阵阵低沉的咆哮声。他知道，这声音来自城堡深处的那条巨龙。

就在李明即将到达巨龙的巢穴时，突然，一阵刺耳的警报声在他耳边响起。

“警告！系统异常！即将强制退出游戏！”李明心中一惊，还没等他反应过来，眼前的景象瞬间消失，他回到了现实世界。

“怎么回事？”李明摘下头盔，心中充满了疑惑。他从未遇到过这样的情况，难道是设备出了问题？他决定联系游戏的客服，询问具体情况。

然而，当他打开电脑，准备登录游戏官网时，却发现整个网络都陷入了瘫痪。不仅是“幻界”，所有的在线游戏和服务都无法连接。

“这到底是怎么回事？”李明感到一阵不安，仿佛有什么大事即将发生。

就在这时，他的手机突然响了起来。来电显示是一个陌生的号码。

“喂，你好。”李明接通了电话。

“

2024-11-28 14:00:45.734 | WARNING  | metagpt.utils.cost_manager:update_cost:49 - Model glm-4-plus not found in TOKEN_COSTS.
2024-11-28 14:00:45.736 | INFO     | __main__:_act:174 - student : **第一章：幻界迷踪**

公元2045年，科技的发展已经超越了人类的想象。虚拟现实技术不再是遥不可及的梦想，而是成为了人们日常生活的一部分。在这个时代，最受欢迎的虚拟现实游戏名为“幻界”。

“幻界”是一款全球联网的大型多人在线角色扮演游戏（MMORPG），它的独特之处在于其高度逼真的虚拟世界和几乎无限制的自由度。玩家可以通过神经接口设备进入游戏，体验一个完全不同的平行世界。

李明，一个普通的大学生，也是“幻界”的忠实玩家。他从小就对游戏充满了热情，尤其是对那些能够让他逃离现实、沉浸其中的虚拟世界。今天，他像往常一样，戴上了神经接口头盔，准备进入“幻界”。

“欢迎来到幻界，勇士！”系统的提示音响起，李明的眼前一黑，随后便置身于一个充满奇幻色彩的世界。他站在一片广袤的草原上，远处是巍峨的山脉和神秘的森林，天空中飘浮着几朵洁白的云。

李明操控着自己的角色，一个身穿银色铠甲的骑士，开始了今天的冒险。他的目标是完成一个名为“龙之宝藏”的任务，这个任务据说奖励丰厚，但也异常危险。

在前往任务地点的路上，李明遇到了各种各样的玩家和NPC（非玩家角色）。有的玩家热情地与他打招呼，有的则冷眼相对，甚至还有人试图抢夺他的装备。李明小心翼翼地应对着，他知道在这个世界里，信任和警惕同样重要。

终于，李明来到了任务地点——一座被浓雾笼罩的古老城堡。城堡的大门紧闭，上面刻满了神秘的符文。李明深吸一口气，推开了大门，一股阴冷的气息扑面而来。

城堡内部昏暗而寂静，只有墙壁上摇曳的火把发出微弱的光芒。李明小心翼翼地走在长长的走廊上，耳边传来阵阵低沉的咆哮声。他知道，这声音来自城堡深处的那条巨龙。

就在李明即将到达巨龙的巢穴时，突然，一阵刺耳的警报声在他耳边响起。

“警告！系统异常！即将强制退出游戏！”李明心中一惊，还没等他反应过来，眼前的景象瞬间消失，他回到了现实世界。

“怎么回事？”李明摘下头盔，心中